# Vanilla GAN for MNIST data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [2]:
# Model Input dimension
img_rows = 28
img_cols = 28
channels = 1
# Input image dimensions
img_shape = (img_rows, img_cols, channels)

# Size of noise vector, used as input to the generator
z_dim = 100

In [3]:
# Build Generator
def build_generator(img_shape, z_dim):
    model = Sequential()
    # A fully connected layer
    model.add(Dense(128, input_dim=z_dim, activation='relu')) 
    # Output with tanh activation
    model.add(Dense(28 * 28 * 1, activation='tanh'))
    # Reshape the output to Image dimension
    model.add(Reshape(img_shape))
    return model
# Build Discriminator
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    # A fully connected layer
    model.add(Dense(128, activation = 'relu'))
    # Output is the probabitly
    model.add(Dense(1, activation='sigmoid'))
    return model

In [4]:
# Build model

def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Build and complie the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])
# Built generator
generator = build_generator(img_shape, z_dim)
# Keeps Discriminator’s parameters constant for Generator training
discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

In [5]:
# Output sample images
def sample_images(generator, image_grid_rows=4, image_grid_columns=4):
    # Sample random noise
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))
    # Generate Images from random noise
    gen_imgs = generator.predict(z)
    # Rescale to [0,1]
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    # Set image grid
    fig, axs = plt.subplots(image_grid_rows,
                            image_grid_columns,
                            figsize=(4, 4),
                            sharey=True,
                            sharex=True)
    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1

In [8]:
# Train GAN model
losses = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):
    # Load data set
    (X_train, _), (_, _) = mnist.load_data()
    # Rescale to [-1,1]
    X_train = X_train / 127.5 - 1.0
    X_train = np.expand_dims(X_train, axis=3)
    # Label for real and face data
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    # Number of epochs
    for iteration in range(iterations):
        
        # 1. Train the Discriminator
        # ----
        # 1.a. Get a random batch of real images: x
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        
        # 1.b.Generates a batch of fake images G(z)=x*
        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)
        
        # 1.c Compute the classification losses for D(x) and D(x*)
        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        # take the average of losses of real and fake training
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        
        # 2. Train the generator
        # ----
        # 2.a Generate a batch of fake image G(z) = x*
        # from random noise vector z
        z = np.random.normal(0, 1, (batch_size, 100))
        gen_samples = generator.predict(z) 
        
        # 2.b Train the generator
        g_loss = gan.train_on_batch(z, real) # why z?
        
        # Save the losses and accs to plot them after training
        if (iteration + 1) % sample_interval == 0:
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
            (iteration + 1, d_loss, 100.0 * accuracy, g_loss))
            # Output a sample of generated Images
            #sample_images(generator)

In [9]:
# Running the model
iterations = 500
batch_size = 128
sample_interval = 100
train(iterations, batch_size, sample_interval)

100 [D loss: 0.255900, acc.: 90.23%] [G loss: 5.423917]
200 [D loss: 0.150835, acc.: 92.97%] [G loss: 4.410271]
300 [D loss: 0.198588, acc.: 89.84%] [G loss: 4.978428]
400 [D loss: 0.237980, acc.: 89.84%] [G loss: 3.789958]
500 [D loss: 0.111535, acc.: 94.53%] [G loss: 4.291515]
